# Анализ вакансий на LinkedIn
Итак, на вход есть датасет с вакансиями с сайта LinkedIn, который необходимо обработать для дальнейшей визуализации информации о рынке вакансий для аналитиков (дата аналитики и BI аналитики) в Европе
- ознакомиться с данными
- провести предобработку (дубликаты, пропуски)
- выделить хард-скиллы
- отфильтровать нерелевантные вакансии
- сформировать датасеты, подходящие для визуализации, только с необходимыми параметрами

По итогу, по отобранной информации (в отдельном датасете) требуется собрать дашборд собрать, где будут отражены:
- количество вакансий по странам (в относительных значениях)
- список нанимающих компаний с указанием количества вакансий 
- количество вакансий по типу занятости
- ТОП-10 сфер деятельности компаний
- размер компаний и количество вакансий  
Дополнительно должны быть фильтры по странам и типу занятости, а также индикатор по общему количеству вакансий

In [1]:
# pip install Levenshtein

дополнительно в процессе подгружалась таблица с переводами отдельных фраз и слов на другие языки (довольно грубая). Также загружена на гугл-диск (https://docs.google.com/spreadsheets/d/1F9hM6azZrXjyNl23RrF_P9tKkt-2ILjq/edit?usp=drive_link&ouid=100434527249103277012&rtpof=true&sd=true)

ссылка на дашборд
(https://public.tableau.com/app/profile/marat.pshikhachev/viz/linkedIn_dashboard/Dashboard_LinkedIn?publish=yes)


In [2]:
import pandas as pd
import numpy as np
from Levenshtein import distance as lev
import re

In [3]:
# загрузим данные
df = pd.read_csv('vacancy_linkedIn.csv')
df.head(20)

,title,location,country,employment_type,company_name,employee_qty,company_field,skills,job_description,applicants
0,Data Analyst,Basel,Switzerland,On-site,PharmiWeb.Jobs: Global Life Science Jobs,11-50 employees,Staffing and Recruiting,None,What You Will Achi...,47.0
1,Data Analyst - Logistics,Coventry,United Kingdom,On-site,Resolute Recruitment,not specified,not specified,None,,NaN
2,Data Analyst - Logistics,Coventry,United Kingdom,On-site,Resolute Recruitment,not specified,not specified,None,Data Analyst - Lo...,NaN
3,Data Analyst (Space & Planning),South Molton,United Kingdom,On-site,Mole Valley Farmers,not specified,not specified,None,Salary: To b...,NaN
4,Data Analyst,Lugano,Switzerland,On-site,FORFIRM,not specified,not specified,None,FORFIRM is p...,NaN
5,Data Analyst - Logistics,Southampton,United Kingdom,On-site,"Butler, Bridge & May",not specified,not specified,None,Location: Southam...,NaN
6,Data Analyst,Leeds,United Kingdom,On-site,Maria Mallaband Care Group Ltd,not specified,not specified,None,We’re Maria Malla...,NaN
7,Data Analyst,Nuneaton,United Kingdom,Hybrid,Kelly Group,not specified,not specified,None,Kelly Group are s...,NaN
8,Data Analyst,Paris,France,On-site,eXalt,"501-1,000 employees",IT Services and IT Consulting,"<span class=""visually-hidden""><!-- -->Skills: ...",Qui sont-ils ? ...,140.0
9,Data Analyst - Hybrid Working,Cambridge,United Kingdom,On-site,Blue Arrow,not specified,not specified,None,Data Analyst ...,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 998 entries, 0 to 997
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   title            998 non-null    object 
 1   location         998 non-null    object 
 2   country          998 non-null    object 
 3   employment_type  998 non-null    object 
 4   company_name     996 non-null    object 
 5   employee_qty     998 non-null    object 
 6   company_field    998 non-null    object 
 7   skills           998 non-null    object 
 8   job_description  998 non-null    object 
 9   applicants       838 non-null    float64
dtypes: float64(1), object(9)
memory usage: 78.1+ KB


Итак, имеется 10 столбцов:
- `title` - название вакансии - на этапе предобработки -перевести все наименования в нижний регистр (на всякий случай)
- `location` и `country` - пропусков нет. Проверить на наличие неявных дубликатов (особенно в country).
- `employment_type` - тип занятости - проверить на дубликаты.
-`company_name` - название компании. Проверить на пропуски (как минимум у двух компаний нет названия) и неявные дубликаты
- `employee_qty` - количество сотрудников. Проверить сколько компаний не указали количество сотрудников. Проверить категории на неявные дубликаты.
- `company_field` - область деятельности. Проверить категории и неявные дубликаты. Посмотреть пропуски.
- `skills` и `job_description` - перевести в нижний регистр для дальнейшего поиска скиллов. Проверить сколько пропусков в каждом.
- `applicants` - количество кандидатов. Проверить сколько пропусков. и Нет ли аномальных значений.


## Предобработка

### Общая проверка на дубликаты

In [5]:
df.duplicated().sum()

112

Итак, имеется 112 полных дубликатов. Удалим всё лишнее.

In [6]:
df = df.drop_duplicates().reset_index(drop=True) 

In [7]:
df.duplicated().sum()

0

Дубликаты удалены.

### Столбец `title` название вакансии
Чтобы проще было в дальнейшем проверить релевантность вакансий, переведём всё в нижний регистр

In [8]:
df['title'] = df['title'].apply(lambda x: x.lower())

Сделано. дополнительно заменим все дефисы на пробелы (если такие есть), чтобы было проще потом отбирать релевантные вакансии 

In [9]:
df['title'] = df['title'].apply(lambda x: x.replace("-", " "))

### Столбец `location` (населенный пункт, регион, локации) и `country` (страна)
Проверим, какие  и страны входят в указанные столбцы 

In [10]:
print(df['location'].nunique())
sorted(df['location'].unique())


428


["'s-Hertogenbosch",
 'Aalborg',
 'Ahlen',
 'Aix-en-Provence',
 'Albuzzano',
 'Alsónémedi',
 'Amersfoort',
 'Amstelveen',
 'Amsterdam',
 'Amsterdam Area',
 'Ancona',
 'Arcole',
 'Arconate',
 'Arluno',
 'Arnhem',
 'Arrasate / Mondragón',
 'Athens',
 'Athens Metropolitan Area',
 'Augusta',
 'Baierbrunn',
 'Barcelona',
 'Bari',
 'Basel',
 'Basingstoke',
 'Bath',
 'Belas',
 'Belfast',
 'Bellinzago Lombardo',
 'Bergamo',
 'Bergen op Zoom',
 'Bergisch Gladbach',
 'Berlin',
 'Berlin Metropolitan Area',
 'Bernay',
 'Bertrange',
 'Bilbao-Bilbo',
 'Binasco',
 'Birkirkara',
 'Blackpool',
 'Boadilla del Monte',
 'Bodelshausen',
 'Bois-Colombes',
 'Bollate',
 'Bologna',
 'Bonn',
 'Bordeaux',
 'Boulogne-Billancourt',
 'Bracknell',
 'Braine-l’Alleud',
 'Bremen',
 'Brindisi',
 'Bristol',
 'Brunswick',
 'Brussels',
 'Brussels Metropolitan Area',
 'Brussels Region',
 'Bucharest',
 'Buckinghamshire',
 'Budapest',
 'Bulgaria',
 'Busto Arsizio',
 'Bègles',
 'Cagliari',
 'Calvignasco',
 'Cambridge',
 'Campo

In [11]:
# тоже самое по странам
print(df['country'].nunique())
sorted(df['country'].unique())

72


[' Austria',
 ' Belgium',
 ' Bulgaria',
 ' Croatia',
 ' Czechia',
 ' Denmark',
 ' Estonia',
 ' Finland',
 ' France',
 ' Germany',
 ' Greece',
 ' Hungary',
 ' Ireland',
 ' Italy',
 ' Latvia',
 ' Lithuania',
 ' Luxembourg',
 ' Malta',
 ' Monaco',
 ' Netherlands',
 ' Norway',
 ' Poland',
 ' Portugal',
 ' Romania',
 ' Slovakia',
 ' Spain',
 ' Sweden',
 ' Switzerland',
 ' United Kingdom',
 'Amsterdam Area',
 'Athens Metropolitan Area',
 'Berlin Metropolitan Area',
 'Brussels Metropolitan Area',
 'Bulgaria',
 'Cologne Bonn Region',
 'Copenhagen Metropolitan Area',
 'Denmark',
 'Eindhoven Area',
 'Finland',
 'France',
 'Germany',
 'Greater Banska Bystrica Area',
 'Greater Barcelona Metropolitan Area',
 'Greater Dijon Area',
 'Greater Lyon Area',
 'Greater Madrid Metropolitan Area',
 'Greater Milan Metropolitan Area',
 'Greater Munich Metropolitan Area',
 'Greater Munster Area',
 'Greater Nuremberg Metropolitan Area',
 'Greater Oslo Region',
 'Greater Palma de Mallorca Metropolitan Area',
 'Gr

In [12]:
# во-первых пройдёмся по строкам и удалим лишние пробелы (по краям), если они есть (заодно и у стран).
df['location'] = df['location'].apply(lambda x: x.strip())
df['country'] = df['country'].apply(lambda x: x.strip())

Бросается в глаза, что в столбце со страной есть позиции, которые были и в столбце с локацией. Посчитаем такие строки добавив столбец с индикатором.

In [13]:
df['indicator'] = df.apply(lambda x: 1 if x['location'] == x['country'] else 0, axis = 1)
df['indicator'].sum()

103

103 - много. Посмотрим, что это за строки.

In [14]:
df[df['indicator'] == 1].head()

,title,location,country,employment_type,company_name,employee_qty,company_field,skills,job_description,applicants,indicator
31,business intelligence analyst,Italy,Italy,Remote,Nexi Digital,51-200 employees,Financial Services,"<span class=""visually-hidden""><!-- -->Skills: ...",Come join us and ...,NaN,1
39,data analyst,Luxembourg,Luxembourg,On-site,BCEE,"1,001-5,000 employees",Banking,"<span class=""visually-hidden""><!-- -->Skills: ...",En vue de renforc...,73.0,1
45,data analyst,Brussels Metropolitan Area,Brussels Metropolitan Area,Hybrid,Nexeo,201-500 employees,IT Services and IT Consulting,"<span class=""visually-hidden""><!-- -->Skills: ...",Nexeo is current...,NaN,1
46,bi analyst,Portugal,Portugal,Remote,Devoteam,"5,001-10,000 employees",Information Technology & Services,"<span class=""visually-hidden""><!-- -->Skills: ...",About Devoteam ...,88.0,1
48,data analyst,Greater Paris Metropolitan Region,Greater Paris Metropolitan Region,Hybrid,Echo Analytics,11-50 employees,"Technology, Information and Internet","<span class=""visually-hidden""><!-- -->Skills: ...",We’re on a missio...,NaN,1


В некоторых случаях в столбцах дублируется страна, а в некоторых - сама локация. Пойдём по порядку. Сначала пройдёмся по локациям

Бросается в глаза, что в локации в некоторых случаях указан только сам город, а в некоторых регион или агломерация, которая сформирована вокруг города. Возникает вопрос, стоит ли заменить аглмерацию на сам центр - вероятно в масштабах Европы (наш исследуемый регион) - стоит. Поэтому уберём из ячеек слова 'Region', 'Area', 'Metropolitan', 'Greater'

In [15]:
def delete_words(string):
    list_words = ['Region', 'Area', 'Metropolitan', 'Greater']
    for word in list_words:
        if word in string:
            string = string.replace(word, '')
    string = string.strip()
    return string

In [16]:
# сделаем столбец только с городом (изначальный не трогаем на всякий случай)
df['city'] = df['location'].apply(delete_words)
df['city'].nunique()

407

В новом столбце таким образом оказалось обработано 22 наименования локаций.
Кроме того, можно заметить, что некоторые города написаны на нескольких языках. А также ещё в некоторых случаях есть разные варианты написаний городов (Франкфурт и Франкфурт-на-Майне).  
Но на некоторое время вернёмся в столбец со странами - там дублировались эти агломерации. Попробуем их поправить.


In [17]:
#  сделаем ещё один столбец с индикатором, который отразит наличие изменений в столбце 'city' по сравнению со столбцом 'location', где были пересечения со столбцом 'country'
def changed(row):
    if row['indicator'] == 1:
        if row['location'] != row['city']:
            return 1
        else:
            return 0
    else:
        return 0 
        

df['changed_ind'] = df.apply(changed, axis=1)

In [18]:
df['changed_ind'].sum()

47

47 строк, где агломерации, регионы и районы указаны вместо стран. Далее выделим из этих строк города, сгруппируем по ним и посмотрим, сколько стран приходится на каждый город.

In [19]:
cities_country = df[df['changed_ind'] == 1]['city'].unique()
cities_country_grouped = (
    df[df['city'].isin(cities_country)]
    .groupby('city', as_index = False)
    .agg({'country': 'nunique'})
    .sort_values(by='country', ascending=False)
)
cities_country_grouped

,city,country
0,Amsterdam,2
12,Lisbon,2
26,Warsaw,2
24,Stuttgart,2
23,Prague,2
21,Paris,2
19,Oslo,2
18,Nuremberg,2
16,Munich,2
15,Milan,2


In [20]:
print('количество городов, у которых можно пределить страну в датасете:', cities_country_grouped[cities_country_grouped['country'] == 2]['city'].count())
print('количество городов, у которых нельзя пределить страну в датасете по альтернативным строкам:', cities_country_grouped[cities_country_grouped['country'] == 1]['city'].count())

количество городов, у которых можно пределить страну в датасете: 19
количество городов, у которых нельзя пределить страну в датасете по альтернативным строкам: 9


Итак, есть 19 городов, у которых в некоторых строках вместо страны указана агломерация или регион и у которых можно определить страну по датасету. У оставшихся 9 городов с такой проблемой определить страну простым способом нельзя (их всего 9 поэтому сделаем вручную для экономии времени)

In [21]:
# список городов для определения страны
cities_list = cities_country_grouped[cities_country_grouped['country'] == 2]['city']

In [22]:
# у 19 городов себерём страны и сделаем словарь
country_city_grouped = (
    df.loc[(df['city'].isin(cities_list)) & (df['indicator'] == 0)]
    .groupby(['country', 'city'], as_index=False)
    .agg({'location': 'nunique'})
)

dict_city_country ={}
for i in list(country_city_grouped.index):
    dict_city_country[country_city_grouped.loc[i, 'city']] = country_city_grouped.loc[i, 'country']
dict_city_country

{'Brussels': 'Belgium',
 'Prague': 'Czechia',
 'Copenhagen': 'Denmark',
 'Dijon': 'France',
 'Lyon': 'France',
 'Paris': 'France',
 'Berlin': 'Germany',
 'Munich': 'Germany',
 'Nuremberg': 'Germany',
 'Stuttgart': 'Germany',
 'Athens': 'Greece',
 'Milan': 'Italy',
 'Amsterdam': 'Netherlands',
 'Eindhoven': 'Netherlands',
 'Oslo': 'Norway',
 'Warsaw': 'Poland',
 'Lisbon': 'Portugal',
 'Barcelona': 'Spain',
 'Madrid': 'Spain'}

In [23]:
# недостающие 9 добавим вручную (наверно можно было бы автоматизировать с помошю парсинга, в данном случае так быстрее)
# повторно выведем эти 9 городов
cities_country_grouped[cities_country_grouped['country'] == 1]['city']

11               Krakow
10                 Iasi
17              Munster
6          Cologne Bonn
20    Palma de Mallorca
22                  Pau
25               Verona
2       Banska Bystrica
27              Wroclaw
Name: city, dtype: object

In [24]:
# дополняем словарь
dict_city_country['Krakow'] = 'Poland'
dict_city_country['Iasi'] = 'Romania'
dict_city_country['Munster'] = 'Germany'
dict_city_country['Cologne Bonn'] = 'Germany'
dict_city_country['Palma de Mallorca'] = 'Spain'
dict_city_country['Pau'] = 'France'
dict_city_country['Verona'] = 'Italy'
dict_city_country['Banska Bystrica'] = 'Slovakia'
dict_city_country['Wroclaw'] = 'Poland'
dict_city_country

{'Brussels': 'Belgium',
 'Prague': 'Czechia',
 'Copenhagen': 'Denmark',
 'Dijon': 'France',
 'Lyon': 'France',
 'Paris': 'France',
 'Berlin': 'Germany',
 'Munich': 'Germany',
 'Nuremberg': 'Germany',
 'Stuttgart': 'Germany',
 'Athens': 'Greece',
 'Milan': 'Italy',
 'Amsterdam': 'Netherlands',
 'Eindhoven': 'Netherlands',
 'Oslo': 'Norway',
 'Warsaw': 'Poland',
 'Lisbon': 'Portugal',
 'Barcelona': 'Spain',
 'Madrid': 'Spain',
 'Krakow': 'Poland',
 'Iasi': 'Romania',
 'Munster': 'Germany',
 'Cologne Bonn': 'Germany',
 'Palma de Mallorca': 'Spain',
 'Pau': 'France',
 'Verona': 'Italy',
 'Banska Bystrica': 'Slovakia',
 'Wroclaw': 'Poland'}

In [25]:
# теперь заменим название регионов и агломераций в столбце 'country' на коректные страны
df['country'] = df.apply(lambda x: dict_city_country[x['city']] if (x['changed_ind'] == 1) else x['country'], axis = 1)
print(df['country'].nunique())
df['country'].unique()

30


array(['Switzerland', 'United Kingdom', 'France', 'Netherlands',
       'Ireland', 'Poland', 'Hungary', 'Greece', 'Italy', 'Sweden',
       'Lithuania', 'Belgium', 'Luxembourg', 'Germany', 'Portugal',
       'Bulgaria', 'Spain', 'Czechia', 'Latvia', 'Estonia', 'Malta',
       'Austria', 'Romania', 'Slovakia', 'Norway', 'Denmark', 'Finland',
       'Rotterdam and The Hague', 'Monaco', 'Croatia'], dtype=object)

Итак, в столбце страна остались почти только страны, но есть странная запись 'Rotterdam and The Hague' - аэропорт Роттердама. Это тоже нужно исправить. 

In [26]:
# пока вручную.
df['country'] = df['country'].replace('Rotterdam and The Hague', 'Netherlands')
df['city'] = df['city'].replace('Rotterdam and The Hague', 'Rotterdam')


Исправлено. Итого у нас 29 стран Европы. Посмотрим, сколько в итоге стран.

In [27]:
print(df['country'].nunique())
sorted(df['country'].unique())

29


['Austria',
 'Belgium',
 'Bulgaria',
 'Croatia',
 'Czechia',
 'Denmark',
 'Estonia',
 'Finland',
 'France',
 'Germany',
 'Greece',
 'Hungary',
 'Ireland',
 'Italy',
 'Latvia',
 'Lithuania',
 'Luxembourg',
 'Malta',
 'Monaco',
 'Netherlands',
 'Norway',
 'Poland',
 'Portugal',
 'Romania',
 'Slovakia',
 'Spain',
 'Sweden',
 'Switzerland',
 'United Kingdom']

Возвращаемся к проблеме разного написания городов.  
Сначала попробуем обработать города, которые пишутся в коротком и полном варианте (пример: Франкфурт и Франкфурт-на-Майне).


In [28]:
#  пройдёмся по каждой стране, внутри страны составим список городов и сравним города друг сдругом на предмет вхождения подстроки в полное название

# словарь для замены
city_full_name_change = {}
double_cities = [] # сюда положим агломерации, в которые входят несколько городов

# ситуаци, когда агломерация вкллючает несколько крупных городов (Кельн-Бонн), трогать пока не будем.
# будем записывать их отдельно
for country in df['country'].unique():
    cities = df[df['country'] == country]['city'].unique()
    i = 0
    verified_idx = []
    while i < len(cities):
        if cities[i] != country:
            if i not in verified_idx:
                j = 0
                while j < len(cities):
                    if j != i:
                        if cities[i] in cities[j]:
                            if j not in verified_idx:
                                city_full_name_change[cities[j]] = cities[i]
                                verified_idx.append(j)
                            else:
                                city_full_name_change.pop(cities[j])
                                double_cities.append(cities[j]) 
                    j += 1
        i += 1
            
print(double_cities)
print(len(city_full_name_change))
city_full_name_change

['Cologne Bonn']
13


{'Issy-les-Moulineaux': 'Moulineaux',
 'County Dublin': 'Dublin',
 'Dublin City': 'Dublin',
 'County Cork': 'Cork',
 'Nova Milanese': 'Milan',
 'Cusano Milanino': 'Milan',
 'San Donato Milanese': 'Milan',
 'Novate Milanese': 'Milan',
 'Carpiano': 'Carpi',
 'Stockholm County': 'Stockholm',
 'Ottignies-Louvain-la-Neuve': 'Louvain-la-Neuve',
 'Frankfurt am Main': 'Frankfurt',
 'Palma de Mallorca': 'Palma'}

Сбор полных и неполных названий прошёл не идеально:
- Issy-les-Moulineaux на самом деле пригород Парижа, а Moulineaux находится в другом месте
- Carpiano - пригород Милана, а Carpi - другой город.  

Кроме того, есть локация 'Cologne Bonn', к которой относятся сразу два города в списке: Кёльн и Бонн.
Посмотрим, сколько строк охватывают эти населённые пункты.


In [29]:
len(df[df['city'].isin(list(city_full_name_change.keys()) + double_cities)])

22

всего 22 строки. Исходя из того, что в большинстве случаев указанные нас. пункты или локации привязаны к более крупным городам (по сути их ближайшие пригороды), то в масштабах целой Европы и небольшого количества охватываемых строк мы можем заменить их на эти города (предварительно вручную исправим ошибки). 'Cologne Bonn' ввиду неопределённости трогать не будем.

In [30]:
len(df[df['city'].isin(list(city_full_name_change.keys()))])

21

Итого, можем обработать 21 строку.

In [31]:
city_full_name_change['Issy-les-Moulineaux'] = 'Paris'
city_full_name_change['Carpiano'] = 'Milan'

df['city'] = df['city'].apply(lambda x: city_full_name_change[x] if x in city_full_name_change.keys() else x)
len(df[df['city'].isin(list(city_full_name_change.keys()))])

0

Готово. Теперь похожим способом разберёмся с названиями на разных языках.

In [32]:
#  сделаем следующим образом: в рамках страны сверим названия города с последуюшщими по расстоянию Левенштейна 
#  если отношение этого растояния к первому названию будет меньше 0.35, то заменим проверяемое на первое 

# словарь для замены
city_change = {}

# будем записывать их отдельно
for country in df['country'].unique():
    cities = df[df['country'] == country]['city'].unique()
    i = 0
    verified_idx = []
    while i + 1 < len(cities):
        if cities[i] != country:
            if i not in verified_idx:
                j = i + 1
                while j < len(cities):
                    lev_distance = lev(cities[i], cities[j])
                    k = lev_distance / len(cities[i])
                    if  k <= 0.35:
                        if j not in verified_idx:
                            city_change[cities[j]] = cities[i]
                            verified_idx.append(j)
                        else:
                            if k < lev(city_change[cities[j]], cities[j]) / len(city_change[cities[j]]):
                                city_change[cities[j]] = cities[i]
                    j += 1
        i += 1
            
print(len(city_change))
city_change


13


{'Southampton': 'South Molton',
 'Vannes': 'Nantes',
 'Vitry-sur-Seine': 'Neuilly-sur-Seine',
 'Rotterdam': 'Amsterdam',
 'Krakow': 'Cracow',
 'Wrocław': 'Wroclaw',
 'Gessate': 'Cesate',
 'Rosate': 'Cesate',
 'Vanzago': 'Inzago',
 'San Colombano al Lambro': 'San Zenone al Lambro',
 'Münster': 'Munster',
 'Lisboa': 'Lisbon',
 'Kosice': 'Košice'}

В сформированным грубым способом список городов, которые подобным образом написаны дважды на другом языке, попало 13 городов. Из них реальные пары - 5шт (Cracow, Wroclaw, Munster, Lisbon, Košice). Остальные - ошибка. Сделаем замену только по реальным.

In [33]:
for key in list(city_change.keys()):
    if key not in ['Krakow', 'Wrocław', 'Münster', 'Lisboa', 'Kosice']:
        city_change.pop(key)
        
len(df[df['city'].isin(list(city_change.keys()))])



7

Так можно обработать ещё 7 строк.

In [34]:
df['city'] = df['city'].apply(lambda x: city_change[x] if x in city_change.keys() else x)
len(df[df['city'].isin(list(city_change.keys()))])


0

Обработано.  
У нас осталось проблема, когда в графе город указана страна. Посмотрим на эти случаи.

In [35]:
# у нас есть поле индикатор на такой случай обновим его и выведем строки, где только страны.
df['indicator'] = df.apply(lambda x: 1 if x['city'] == x['country'] else 0, axis=1)

df[df['indicator'] == 1]['indicator'].sum()

53

53 строки - довольно много. Но сделать, что-то с ними нельзя. Посмотрим какие страны попали в этот список и сколько их

In [36]:
print(df[df['indicator'] == 1]['country'].nunique())
df[df['indicator'] == 1]['country'].unique()

16


array(['Italy', 'Luxembourg', 'Portugal', 'France', 'Bulgaria', 'Hungary',
       'Poland', 'Germany', 'Denmark', 'Monaco', 'Spain', 'Finland',
       'Romania', 'Netherlands', 'Ireland', 'Lithuania'], dtype=object)

Из указанного списка Luxembourg и Monaco по сути города-государства, т.е противоречий никаких нет. В остальных случаях предлагаю заменить в графе город на пометку "город не указан" ("not specified")

In [37]:
len(df[df['city'].isin(['Luxembourg', 'Monaco'])])

11

Т.е. быть затронуто 42 строки.

In [38]:
df['city'] = df.apply(
    lambda x: 'not specified' if (x['city'] not in ['Luxembourg', 'Monaco'] and x['indicator'] == 1) else x['city'], axis=1
)
# обновим столбец с индикотором и посмотрим, сколько осталось строк, где страна совпадает с городом (должно быть 11)
df['indicator'] = df.apply(lambda x: 1 if x['city'] == x['country'] else 0, axis=1)
print('количество оставшихся строк, где "страна = город":', df[df['indicator'] == 1]['indicator'].sum())

print('количество строк с "not specified":', df[df['city'] == "not specified"]['city'].count())



количество оставшихся строк, где "страна = город": 11
количество строк с "not specified": 42


42 строки обработано (без указания города). 11 оставлены как корректные.

In [39]:
df['city'].nunique()

375

375 - итоговое количество уникальных записей в столбце `город`

### Тип занятости (`employment_type`)
Проверим какие есть значения в этой графе.

In [40]:
df['employment_type'].unique()

array(['On-site', 'Hybrid', 'not specified', 'Remote'], dtype=object)

4 варианта. Каких-то необычных нет. На всякий случай посмотрим на вариант 'not specified', т.е. тип занятости не указан.

In [41]:
df[df['employment_type'] == 'not specified']

,title,location,country,employment_type,company_name,employee_qty,company_field,skills,job_description,applicants,indicator,city,changed_ind
14,data analyst,Craven Arms,United Kingdom,not specified,networx | Recruitment Software & Services by IRIS,not specified,not specified,None,Are you passionat...,NaN,0,Craven Arms,0
66,data analyst,Epsom,United Kingdom,not specified,First Place Listings,1-10 employees,"Technology, Information and Internet",None,This job is...,9.0,0,Epsom,0
67,"data analyst €60, per hour amsterdam based",Amsterdam,Netherlands,not specified,Computer Futures,"501-1,000 employees",IT Services and IT Consulting,"<span class=""visually-hidden""><!-- -->Skills: ...",Data Analyst A...,73.0,0,Amsterdam,0
92,data analyst (engineer),Madrid,Spain,not specified,Revolut,"1,001-5,000 employees",Financial Services,"<span class=""visually-hidden""><!-- -->Skills: ...",About Revolut P...,NaN,0,Madrid,0
99,data analyst,Barcelona,Spain,not specified,Proton | Privacy by Default,201-500 employees,"Technology, Information and Internet","<span class=""visually-hidden""><!-- -->Skills: ...","A better internet,...",NaN,0,Barcelona,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
805,software engineer,Milan,Italy,not specified,Enel Group,"10,001+ employees",Utilities,"<span class=""visually-hidden""><!-- -->Skills: ...",The team Technolog...,29.0,0,Milan,0
813,credit data analyst (product),Moravia-Silesia,Czechia,not specified,Revolut,"1,001-5,000 employees",Financial Services,"<span class=""visually-hidden""><!-- -->Skills: ...",About Revolut P...,27.0,0,Moravia-Silesia,0
821,analyst application development analytics,Székesfehérvár,Hungary,not specified,HARMAN International,"10,001+ employees",Computers and Electronics Manufacturing,"<span class=""visually-hidden""><!-- -->Skills: ...",What We Offer: ...,22.0,0,Székesfehérvár,0
857,data analyst,Warsaw,Poland,not specified,JLL Technologies,"1,001-5,000 employees",IT Services and IT Consulting,"<span class=""visually-hidden""><!-- -->Skills: ...",JLL Technologies i...,21.0,0,Warsaw,0


Вроде ничего необычного. Таких записей 67шт. Достоверно определить какой тип занятости подразумевается у данной вакансии невозможно, т.к. даже в одной компании к разным должностям могут быть разные требования. Поэтому в этом случае ничего делать не будем.

### Название компании `company_name`
Посмотрим, что тут можно проверить.

In [42]:
df['company_name'].isna().sum()

2

Есть две компании, у которых не указаны названия. Выведем их.

In [43]:
df[df['company_name'].isna()]

,title,location,country,employment_type,company_name,employee_qty,company_field,skills,job_description,applicants,indicator,city,changed_ind
656,senior data engineer (f/m/x),Ulm,Germany,On-site,NaN,201-500 employees,Software Development,"<span class=""visually-hidden""><!-- -->Skills: ...",Software in highl...,103.0,0,Ulm,0
681,data analyst (m/w/d),Osnabrück,Germany,Hybrid,NaN,"10,001+ employees",Mechanical Or Industrial Engineering,"<span class=""visually-hidden""><!-- -->Skills: ...",This job is...,2.0,0,Osnabrück,0


По вакансиям по этим компаниям есть достаточно полные данные (обе из Германии, но разные города), поэтому, предлагается просто заменить пропуск на 'not specified'.

In [44]:
df['company_name'] = df['company_name'].fillna('not specified')

In [45]:
df['company_name'].isna().sum()

0

Посмотрим, сколько уникальных компаний в принципе есть в датасете.

In [46]:
df['company_name'].nunique()

670

In [47]:
df['company_name'].value_counts()

CPM Italy                                                44
TELUS International                                      20
Revolut                                                  12
Agoda                                                     6
TELUS International AI Data Solutions                     6
                                                         ..
HillFive                                                  1
IT-Personalberatung Dr. Dienst & Wenzel GmbH & Co. KG     1
iTrust Partnering                                         1
Lupus alpha Asset Management AG                           1
Infoport                                                  1
Name: company_name, Length: 670, dtype: int64

In [48]:
companies = (
    df.groupby('company_name', as_index=False)
    .agg({'title': 'count'})
    .rename(columns={'title': 'vacancies'})
    .sort_values('vacancies', ascending=False)
)
print('более двух вакансий:', companies[companies['vacancies'] > 2]['company_name'].count())
print('до двух вакансий:', companies[companies['vacancies'] <= 2]['company_name'].count())

более двух вакансий: 35
до двух вакансий: 635


669 известных компаний представлено в датасете (+ две или одна компания записана под 'not specified'). В большинстве своем компании разместили 1 или две вакансии (635 компаний). 35 компаний - от 3 и больше. Поиск неявных дубликатов осложняется тем, что компаний слишком много и среди вероятно могут быть компании просто с похожими названиями и разные дочерние структуры. Поэтому в плане дубликатов здесь делать ничего не будем - слишком легко ошибиться. Тем более название компании идёт от профиля компании в LinkedIn, поэтому появления конкретного дубликата маловероятно.

### Количество сотрудников (`employee_qty`)
Посмотрим, какие категории есть и сколько компаний не указало количество сотрудников.

In [49]:
df['employee_qty'].unique()

array(['11-50 employees', 'not specified', '501-1,000 employees',
       '1,001-5,000 employees', '51-200 employees', '10,001+ employees',
       '1-10 employees', '201-500 employees', '5,001-10,000 employees',
       'Retail Apparel and Fashion',
       'See how you compare to 9 applicants. Try Premium for free',
       'Svein Grande is hiring for this job',
       'See how you compare to 19 applicants. Try Premium for free',
       'See how you compare to 22 applicants. Try Premium for free',
       'Romain GUIHENEUF is hiring for this job',
       'See how you compare to 13 applicants. Try Premium for free',
       'See recent hiring trends for Devonshire Hayes Recruitment Specialists Ltd. Try Premium for free',
       'See how you compare to 10 applicants. Try Premium for free',
       'See how you compare to 4 applicants. Try Premium for free'],
      dtype=object)

In [50]:
df['employee_qty'].isna().sum()

0

Основные проблемы:
- есть строки с 'not specified'
- есть строки, где вместо количества сотрудников указана какая-то иная информация (количество заявителей, область деятельности компании и т.д.)

Начнём со второго пункта. Там есть информация о других кандидатах на вакансию. Вероятно эта та информация, которая должно была попасть в графу 'applicants'. Выведем эти строки и только интересующие нас столбцы.


In [51]:
applicants = []
for elem in df['employee_qty'].unique():
    if 'applicants' in elem:
        applicants.append(elem)
df[df['employee_qty'].isin(applicants)][['employee_qty', 'applicants']]

,employee_qty,applicants
285,See how you compare to 9 applicants. Try Premi...,9.0
335,See how you compare to 19 applicants. Try Prem...,19.0
360,See how you compare to 22 applicants. Try Prem...,22.0
583,See how you compare to 13 applicants. Try Prem...,13.0
673,See how you compare to 10 applicants. Try Prem...,10.0
876,See how you compare to 4 applicants. Try Premi...,4.0


Всё верно - графа 'employee_qty' в этих строках оказалась заполнена неверно (эти строки можно также пометить как 'not specified').  
Также есть строки с указанной сферой деятельности - выведем эти строки


In [52]:
df[df['employee_qty'] == 'Retail Apparel and Fashion'][['employee_qty', 'company_field']]

,employee_qty,company_field
220,Retail Apparel and Fashion,Retail Apparel and Fashion


И снова - это просто ошибка (в 'company_field' информация полностью повторяется - тоже пометить 'not specified').

Поправим столбец.

In [53]:
def to_not_specified(cell):
    if 'employees' in cell or 'not specified' in cell:
        return cell
    else:
        return 'not specified'
df['employee_qty'] = df['employee_qty'].apply(to_not_specified)
df['employee_qty'].unique()

array(['11-50 employees', 'not specified', '501-1,000 employees',
       '1,001-5,000 employees', '51-200 employees', '10,001+ employees',
       '1-10 employees', '201-500 employees', '5,001-10,000 employees'],
      dtype=object)

Некорректное изменено. Теперь займёмся 'not specified'. Для начала посмотрим, сколько таких строк.

In [54]:
df[df['employee_qty'] == 'not specified']['title'].count()

27

Таких строк 27 - не то, чтобы очень много. Можно посмотреть, нет ли среди этих строк компаний, у которых в других вакансиях количество сотрудников указано.

In [55]:
compaies = df[df['employee_qty'] == 'not specified']['company_name'].unique()
len(compaies)

23

Таких компаний 23. Есть ли вакансии, где у них указан штат. 

In [56]:
df.loc[(df['company_name'].isin(compaies)) & (df['employee_qty'] != 'not specified')]

,title,location,country,employment_type,company_name,employee_qty,company_field,skills,job_description,applicants,indicator,city,changed_ind
0,data analyst,Basel,Switzerland,On-site,PharmiWeb.Jobs: Global Life Science Jobs,11-50 employees,Staffing and Recruiting,None,What You Will Achi...,47.0,0,Basel,0
20,data analyst (space & planning),South Molton,United Kingdom,On-site,networx | Recruitment Software & Services by IRIS,51-200 employees,Staffing and Recruiting,None,Salary: To be di...,13.0,0,South Molton,0
22,data analyst,Solihull,United Kingdom,On-site,Mexa Solutions,11-50 employees,Staffing and Recruiting,None,Hybrid - West Mi...,NaN,0,Solihull,0
23,data analyst ii,Dublin,Ireland,On-site,PharmiWeb.Jobs: Global Life Science Jobs,11-50 employees,Staffing and Recruiting,"<span class=""visually-hidden""><!-- -->Skills: ...",Title: Senior QA A...,30.0,0,Dublin,0


Отлично! есть несколько компаний, где в других случаях указано количество сотрудников. Т.е. часть строк, где не указано штат, можно поправить.

In [57]:
companies_staff = (
    df.loc[(df['company_name'].isin(compaies)) & (df['employee_qty'] != 'not specified')]
    .groupby(['company_name', 'employee_qty'], as_index=False)
    .agg({'title': 'count'})
)
companies_staff

,company_name,employee_qty,title
0,Mexa Solutions,11-50 employees,1
1,PharmiWeb.Jobs: Global Life Science Jobs,11-50 employees,2
2,networx | Recruitment Software & Services by IRIS,51-200 employees,1


In [58]:
# сделаем словарь  "компания - штат"
staff_comp = {}
for company, staff in zip(companies_staff['company_name'], companies_staff['employee_qty']):
    staff_comp[company] = staff


In [59]:
# в графе "employee_qty" заменим "not specified" на категорию кол-ва сотрудников, где это возможно.
df['employee_qty'] = (
    df.apply(lambda x:
             staff_comp[x['company_name']] 
             if (x['employee_qty'] == 'not specified' and x['company_name'] in staff_comp.keys())
             else x['employee_qty'],
             axis = 1)
)

In [60]:
df[df['employee_qty'] == 'not specified']['title'].count()

23

Таким образом удалось исправить 4 строки (не слишком густо, но хоть что-то)

### Стобец "сфера деятельности" (`company_field`)
Проверим, есть ли в этом столбце какие-то ненормальные значения или пропуски

In [61]:
# для начала переведём в нижний реестр
df['company_field'] = df['company_field'].apply(lambda x: x.lower())

In [62]:
print(df['company_field'].nunique())
fields = sorted(df['company_field'].unique())
fields

122


['1,001-5,000 employees',
 '1-10 employees',
 '11-50 employees',
 '201-500 employees',
 '501-1,000 employees',
 '51-200 employees',
 'advertising services',
 'airlines and aviation',
 'apparel & fashion',
 'appliances, electrical, and electronics manufacturing',
 'armed forces',
 'automation machinery manufacturing',
 'automotive',
 'aviation and aerospace component manufacturing',
 'banking',
 'biotechnology research',
 'book and periodical publishing',
 'business consulting and services',
 'chemical manufacturing',
 'computer and network security',
 'computer games',
 'computers and electronics manufacturing',
 'construction',
 'consumer goods',
 'consumer services',
 'cosmetics',
 'dairy product manufacturing',
 'defense & space',
 'defense and space manufacturing',
 'education administration programs',
 'entertainment providers',
 'environmental services',
 'farming',
 'financial services',
 'food & beverages',
 'food and beverage manufacturing',
 'food and beverage services',
 'fo

И снова есть фразы, которые попали сюда ошибочно (о колистве сотрудников или количестве соискателей). Проверим их.

In [63]:
#  по соискателям
applicants = []
for elem in df['company_field'].unique():
    if 'applicants' in elem:
        applicants.append(elem)
df[df['company_field'].isin(applicants)][['company_field', 'applicants']]

,company_field,applicants
285,see how you compare to 9 applicants. try premi...,9.0
335,see how you compare to 19 applicants. try prem...,19.0
360,see how you compare to 22 applicants. try prem...,22.0
583,see how you compare to 13 applicants. try prem...,13.0
673,see how you compare to 10 applicants. try prem...,10.0
876,see how you compare to 4 applicants. try premi...,4.0


Информация дублируется.

In [64]:
# по количесвту сотрудников
employees = []
for elem in df['company_field'].unique():
    if 'employees' in elem:
        employees.append(elem)
df[df['company_field'].isin(employees)][['company_field', 'employee_qty']]

,company_field,employee_qty
34,51-200 employees,51-200 employees
44,11-50 employees,11-50 employees
49,11-50 employees,11-50 employees
58,11-50 employees,11-50 employees
61,11-50 employees,11-50 employees
137,51-200 employees,51-200 employees
258,201-500 employees,201-500 employees
282,201-500 employees,201-500 employees
311,51-200 employees,51-200 employees
321,11-50 employees,11-50 employees


И здесь дублируется. 

В графе "company_field" эти значения (а также строкис информацией о том, кто нанимает) можно заменить на 'not specified'

In [65]:
errors = []
for elem in df['company_field'].unique():
    if 'hiring for this job' in elem:
        errors.append(elem)
errors += applicants
errors += employees


df['company_field'] = df['company_field'].apply(lambda x: 'not specified' if x in errors else x)

In [66]:
print(df['company_field'].nunique())

108


Осталось 108 уникальных значений. Лишние убраны.  Дальше можно сделать также, как и в предыдущем пункте - проверить пропуски по компаниям.

In [67]:
df[df['company_field'] == 'not specified']['title'].count()

82

82 строки с пропусками.

In [68]:
compaies = df[df['company_field'] == 'not specified']['company_name'].unique()
len(compaies)

78

Охватывают 78 компаний

In [69]:
(
    df.loc[(df['company_name'].isin(compaies)) & (df['company_field'] != 'not specified')]
    .groupby(['company_name'], as_index=False)
    .agg({'company_field': 'nunique'}).sort_values('company_field', ascending=False)
)

,company_name,company_field
0,Mexa Solutions,1
1,Peroptyx,1
2,PharmiWeb.Jobs: Global Life Science Jobs,1
3,networx | Recruitment Software & Services by IRIS,1


есть 4 компании из тех, у кого в некоторых строках отстутствует информация о сфере деятельности, но есть в других, и при этом сфера деятельности для каждой компании одна.

In [70]:
companies_not_field= (
    df.loc[(df['company_name'].isin(compaies)) & (df['company_field'] != 'not specified')]
    .groupby(['company_name', 'company_field'], as_index=False)
    .agg({'title': 'count'})
)
companies_not_field

,company_name,company_field,title
0,Mexa Solutions,staffing and recruiting,1
1,Peroptyx,it services and it consulting,1
2,PharmiWeb.Jobs: Global Life Science Jobs,staffing and recruiting,2
3,networx | Recruitment Software & Services by IRIS,staffing and recruiting,1


In [71]:
# сделаем словарь  "компания - область деятельности"
fields_comp = {}
for company, fields in zip(companies_not_field['company_name'], companies_not_field['company_field']):
    fields_comp[company] = fields


In [72]:
# в графе "company_field" заменим "not specified" на сферу деятельности, где это возможно.
df['company_field'] = (
    df.apply(lambda x:
             fields_comp[x['company_name']] 
             if (x['company_field'] == 'not specified' and x['company_name'] in fields_comp.keys())
             else x['company_field'],
             axis = 1)
)

In [73]:
# проверяем, сколько осталось пропусков (было 82)
df[df['company_field'] == 'not specified']['title'].count()

77

Итак, удалось убрать 5 пропусков. Можно было бы попробовать выудить из описания к вакансии, но там отдельные слова могут быть вырваны из контекста в процессе поиска, в результате чего сфера деятельности будет определена некорректно. Поэтому больше ничего делать здесь не будем.

Проверим ещё раз на полные дубликаты

In [74]:
df.duplicated().sum()

0

Полных дубликатов нет.

### Столбцы `skills` и `job_description`
По сути, эти столбцы нужны нам, чтобы по каждой вакансии собрать хард-скиллы.  

In [75]:
# предоставленный список скиллов
skills = [
    'a/b testing', 'ab testing', 'actian', 'adobe analytics', 'adobe audience manager',
    'adobe experience platform', 'adobe launch', 'adobe target', 'ai', 'airflow',
    'alooma', 'alteryx', 'amazon machine learning', 'amazon web services', 'aml',
    'amplitude', 'ansible', 'apache camel', 'apache nifi', 'apache spark',
    'api', 'asana', 'auth0', 'aws', 'aws glue', 'azure', 'azure data factory',
    'basecamp', 'bash', 'beats', 'big query', 'bigquery', 'birst', 'bitbucket',
    'blendo', 'bootstrap', 'business objects bi', 'c#', 'c++', 'caffe', 'cassandra',
    'cdata sync', 'chronograf', 'ci/cd', 'cicd', 'clickhouse', 'cloudera', 'cluvio',
    'cntk', 'cognos', 'composer', 'computer vision', 'conda', 'confluence',
    'couchbase', 'css', 'd3.js', 'dash', 'dashboard', 'data factory', 'data fusion',
    'data mining', 'data studio', 'data warehouse', 'databricks', 'dataddo',
    'dataflow', 'datahub', 'dataiku', 'datastage', 'dbconvert', 'dbeaver', 'dbt',
    'deep learning', 'dl/ml', 'docker', 'domo', 'dune', 'dv360', 'dynamodb',
    'elasticsearch', 'elt', 'erwin', 'etl', 'etleap', 'excel', 'facebook business manager',
    'fivetran', 'fuzzy', 'ga360', 'gcp', 'gensim', 'ggplot', 'git', 'github', 'gitlab',
    'google ads', 'google analytics', 'google cloud platform', 'google data flow',
    'google optimize', 'google sheets', 'google tag manager', 'google workspace',
    'grafana', 'hadoop', 'hana', 'hanagrafana', 'hbase', 'hdfs', 'hevo data', 'hightouch',
    'hive', 'hivedatabricks', 'html', 'hubspot', 'ibm coremetrics', 'inetsoft',
    'influxdb', 'informatica', 'integrate.io', 'iri voracity', 'izenda', 'java',
    'java script', 'javascript', 'jenkins', 'jira', 'jmp', 'julia', 'jupyter',
    'k2view', 'kafka', 'kantar', 'kapacitor', 'keras', 'kibana', 'kubernetes',
    'lambda', 'linux', 'logstash', 'looker', 'lstm', 'luidgi', 'matillion', 'matlab',
    'matplotlib', 'mendix', 'metabase', 'microsoft sql', 'microsoft sql server',
    'microstrategy', 'miro', 'mixpanel', 'ml', 'ml flow', 'mlflow', 'mongodb', 'mxnet',
    'mysql', 'natural nanguage processing', 'neo4j', 'nlp', 'nltk', 'nosql', 'numpy',
    'oauth', 'octave', 'omniture', 'omnituregitlab', 'openshift', 'openstack',
    'optimizely', 'oracle', 'oracle business intelligence', 'oracle data integrator',
    'pandas', 'panorama', 'pentaho', 'plotly', 'postgre', 'postgresql', 'posthog',
    'power amc', 'power bi', 'power point', 'powerbi', 'powerpivot', 'powerpoint',
    'powerquery', 'pyspark', 'python', 'pytorch', 'pytorchhevo data', 'qlik',
    'qlik sense', 'qlikview', 'querysurge', 'r', 'raphtory', 'rapidminer', 'redash',
    'redis', 'redshift', 'retool', 'rivery', 'rust', 's3', 'sa360', 'salesforce', 'sap',
    'sap business objects', 'sas', 'sas visual analytics', 'scala', 'scikit-learn',
    'scipy', 'seaborn', 'segment', 'selenium', 'sem rush', 'semrush', 'shell', 'shiny',
    'singer', 'sisense', 'skyvia', 'snowflake', 'spacy', 'spark', 'sparkml', 'splunk',
    'spotfire', 'spreadsheet', 'spss', 'sql', 'ssis', 'sssr', 'stambia', 'statistics',
    'statsbot', 'stitch', 'streamlit', 'streamsets', 'svn', 't-sql', 'tableau', 'talend',
    'targit', 'tealium', 'telegraf', 'tensorflow', 'terraapi', 'terraform', 'theano',
    'thoughtspot', 'timeseries', 'trello', 'unix', 'vba', 'vtom', 'webfocus', 'wfh',
    'xplenty', 'xtract.io', 'yellowfin'
]

In [76]:
# функция для сбора скиллов (выложенный в чате мастерской).
def skills_finder(row, skill_list=skills):
    matched_skills_list=[]
    for i in skill_list:
        if i == 'c++':
            if re.search('\Wc\+\+\W', row['job_description'].lower()):
                matched_skills_list.append(i)
            elif re.search('\Wc\+\+\W', row['skills'].lower()):
                matched_skills_list.append(i)                
        else:
            pattern = (
                r'(\b|\W)'
                + re.escape(i)
                + r'(\b|\W)'
                +'|'
                + r'(\b|\W)'
                +re.escape(i.replace(' ', ''))
                + r'(\b|\W)'
            )
            if re.search(pattern, row['job_description'].lower()):
                matched_skills_list.append(i)
            elif re.search(pattern, row['skills'].lower()):
                matched_skills_list.append(i)        
    return matched_skills_list

In [77]:
df['hard_skills'] = df.apply(skills_finder, axis=1)

In [78]:
df['hard_skills'].head()

0    [data mining, excel, sap, sas, spss, sql, stat...
1                                                   []
2                                                [wfh]
3                                              [excel]
4    [aws, data warehouse, etl, gcp, oracle, oracle...
Name: hard_skills, dtype: object

Первая часть обработки скиллов сделана. Пока этого достаточно. Остальное сделаем отдельно во-второй части работы подготовки к визуализации.

###  Столбец `applicants` - количество кандидатов на вакансию.
Здесь были пропуски. Посмотрим, можно ли что-то с ними сделать.

In [79]:
df['applicants'].isna().sum()

159

159 пропусков - довольно много.  Ранее при проверке других столбцов эта информация могла попасть туда, но в тех случаях в данном столбце пропусков не было. У нас на этот счёт остались не проверены столбцы `skills` и `job_description` - посмотрим их.

In [80]:
counter = 0

for elem in df['skills'].unique():
    if 'applicant' in elem:
        counter += 1
counter

0

в столбце `skills` таких данных нет.

In [81]:
counter = 0
for elem in df['job_description'].unique():
    pattern= (
        r'(\d+\s)'
        + 'applicant'
    )
    if re.search(pattern, elem.lower()):
        counter += 1
counter

0

И в столбце с описанием вакансии количества кандидатов нет.   
Больше эту информацию взять неоткуда, заполнять какими-то средними можно было бы, но скорее некорректно, т.к. это может зависеть от многих факторов. Поэтому этот столбец оставляем без изменений.


## Дополнительная подготовка к визуализации

**Релевантные вакансии**
Нас интересуют только дата-аналитики и Bi-аналитики. КРоме того, нам нужны джуны, т.е. нужно отсеять senior и middle
Наименования вакансий написаны на разных языках, поэтому для проверки нам понадобится дополнительный датасет с переводом интересующих нас должностей на различные языки.

In [82]:
# датасет с переводами был отдельно подготовлен в гугл-таблицах (там есть встроеная функция для перевода)
# коды языков взяты из интернета
try:
    data_analyst = pd.read_excel('https://drive.google.com/uc?export=download&id=1F9hM6azZrXjyNl23RrF_P9tKkt-2ILjq', sheet_name='data_analyst')
    analyst = pd.read_excel('https://drive.google.com/uc?export=download&id=1F9hM6azZrXjyNl23RrF_P9tKkt-2ILjq', sheet_name='analyst')
    BI = pd.read_excel('https://drive.google.com/uc?export=download&id=1F9hM6azZrXjyNl23RrF_P9tKkt-2ILjq', sheet_name='BI')
    data = pd.read_excel('https://drive.google.com/uc?export=download&id=1F9hM6azZrXjyNl23RrF_P9tKkt-2ILjq', sheet_name='data')
    middle_senior = pd.read_excel('https://drive.google.com/uc?export=download&id=1F9hM6azZrXjyNl23RrF_P9tKkt-2ILjq', sheet_name='middle_senior')

except:
# ссылка на гугл диск - https://docs.google.com/spreadsheets/d/1F9hM6azZrXjyNl23RrF_P9tKkt-2ILjq/edit?usp=drive_link&ouid=100434527249103277012&rtpof=true&sd=true
    data_analyst = pd.read_excel('translate.xlsx', sheet_name='data_analyst')
    analyst = pd.read_excel('translate.xlsx', sheet_name='analyst')
    data = pd.read_excel('translate.xlsx', sheet_name='data')
    middle_senior = pd.read_excel('translate.xlsx', sheet_name='middle_senior')


In [83]:
display(data_analyst.head())
display(analyst.head())
display(BI.head())
display(data.head())
display(middle_senior.head())

,basic word,start_language,target_language,result
0,data analyst,en,en,data analyst
1,data analyst,en,af,data -ontleder
2,data analyst,en,am,የውሂብ ተንታኝ
3,data analyst,en,ar,محلل البيانات
4,data analyst,en,az,Məlumat analitiki


,basic word,start_language,target_language,result
0,analyst,en,en,analyst
1,analyst,en,af,ontleder
2,analyst,en,am,ተንታኝ
3,analyst,en,ar,المحلل
4,analyst,en,az,analitik


,basic word,start_language,target_language,result
0,Business intelligence,en,en,Business intelligence
1,Business intelligence,en,af,Sake-intelligensie
2,Business intelligence,en,am,የንግድ ሥራ ብልህነት
3,Business intelligence,en,ar,ذكاء الأعمال
4,Business intelligence,en,az,İşgüzar


,basic word,start_language,target_language,result
0,data,en,en,data
1,data,en,af,data
2,data,en,am,ውሂብ
3,data,en,ar,بيانات
4,data,en,az,məlumat


,basic word,start_language,target_language,result
0,senior,en,en,senior
1,senior,en,af,senior
2,senior,en,am,አዛውንት
3,senior,en,ar,كبير
4,senior,en,az,böyük


In [84]:
data_analyst['result'] = data_analyst['result'].apply(lambda x: x.lower())
analyst['result'] = analyst['result'].apply(lambda x: x.lower())
BI['result'] = BI['result'].apply(lambda x: x.lower())
data['result'] = data['result'].apply(lambda x: x.lower())
middle_senior['result'] = middle_senior['result'].apply(lambda x: x.lower())

In [85]:
# списки (их нужно ещё чуть-чуть почистить и скорректировать)
# кое-что пришлось длавлять вручную
data_analyst_list = data_analyst['result'].to_list()
data_analyst_list += ['data analist', 'analyste des donnees', 'datový analytik']
analyst_list = analyst['result'].to_list()
analyst_list += ['analist', 'analysis', 'analytik', 'analytic']
BI_list = BI['result'].to_list()
data_list = data['result'].to_list()
data_list.append('donnees')
middle_senior_list = []
for elem in middle_senior['result'].to_list():
    if len(elem) > 4:
        middle_senior_list.append(elem)



Проверим в каждой вакансии наличие какого-либо из вариантов.

In [86]:
# #  доп. столбец
# #  cначала оставим только джунов
def relevant_junior(cell, control_list=middle_senior_list):
    if 'junior' in cell:
        return 1
    else:
        i = 0
        result = 1
        while i < len(control_list):
            if control_list[i] in cell:
                result = 0
                i = len(control_list)
            else:
                i += 1
        return result

df['relevant_jun'] = df['title'].apply(relevant_junior)

In [87]:
df[df['relevant_jun'] != 1]['title'].count()

29

29 вакансий не попадающих в junior

In [88]:
# провверяем "data analyst" на разных языках

def relevant_da(row, control_list=data_analyst_list):
    if row['relevant_jun'] == 0:
        return 0
    else:
        i = 0
        result = 0
        while i < len(control_list):
            if control_list[i] in row['title']:
                result = 1
                i = len(control_list)
            else:
                i += 1
    return result

df['relevant_da'] = df.apply(relevant_da, axis=1)

df[df['relevant_da'] == 1].head(10)

,title,location,country,employment_type,company_name,employee_qty,company_field,skills,job_description,applicants,indicator,city,changed_ind,hard_skills,relevant_jun,relevant_da
0,data analyst,Basel,Switzerland,On-site,PharmiWeb.Jobs: Global Life Science Jobs,11-50 employees,staffing and recruiting,None,What You Will Achi...,47.0,0,Basel,0,"[data mining, excel, sap, sas, spss, sql, stat...",1,1
1,data analyst logistics,Coventry,United Kingdom,On-site,Resolute Recruitment,not specified,not specified,None,,NaN,0,Coventry,0,[],1,1
2,data analyst logistics,Coventry,United Kingdom,On-site,Resolute Recruitment,not specified,not specified,None,Data Analyst - Lo...,NaN,0,Coventry,0,[wfh],1,1
3,data analyst (space & planning),South Molton,United Kingdom,On-site,Mole Valley Farmers,not specified,not specified,None,Salary: To b...,NaN,0,South Molton,0,[excel],1,1
4,data analyst,Lugano,Switzerland,On-site,FORFIRM,not specified,not specified,None,FORFIRM is p...,NaN,0,Lugano,0,"[aws, data warehouse, etl, gcp, oracle, oracle...",1,1
5,data analyst logistics,Southampton,United Kingdom,On-site,"Butler, Bridge & May",not specified,not specified,None,Location: Southam...,NaN,0,Southampton,0,"[excel, power bi, powerbi, sap]",1,1
6,data analyst,Leeds,United Kingdom,On-site,Maria Mallaband Care Group Ltd,not specified,not specified,None,We’re Maria Malla...,NaN,0,Leeds,0,"[excel, power bi]",1,1
7,data analyst,Nuneaton,United Kingdom,Hybrid,Kelly Group,not specified,not specified,None,Kelly Group are s...,NaN,0,Nuneaton,0,"[data mining, excel]",1,1
8,data analyst,Paris,France,On-site,eXalt,"501-1,000 employees",it services and it consulting,"<span class=""visually-hidden""><!-- -->Skills: ...",Qui sont-ils ? ...,140.0,0,Paris,0,"[dataiku, nosql, power bi, powerbi, qlikview, ...",1,1
9,data analyst hybrid working,Cambridge,United Kingdom,On-site,Blue Arrow,not specified,not specified,None,Data Analyst ...,NaN,0,Cambridge,0,[],1,1


In [89]:
df['relevant_da'].sum()

428

In [90]:
# теперь посмотрим более сложные варианты, где есть analyst, data, bi в различных комбинациях

def final_relevant(row, analyst=analyst_list, data=data_list, bi = BI_list):
    if row['relevant_da'] == 0 and row['relevant_jun'] == 1:
        i = 0
        result = 0
        while i < len(analyst):
            if analyst[i] in row['title']:
                pattern = (
                    r'(\b|\W|\s|\_)'
                    + 'bi'
                    + r'(\b|\W|\s|\_)'
                )
                if re.search(pattern, row['title']):
                    result = 1
                else:                        
                    j = 0
                    while j < len(data):
                        if data[j] in row['title']:
                            result = 1
                        else:
                            k = 0
                            while k < len(bi):
                                if bi[k] in row['title']:
                                    result = 1
                                    k = len(bi)
                                else:
                                    k += 1
                        if result == 1:
                            j = len(data)
                        else:
                            j += 1
                if result == 1:
                    i = len(analyst)
                else:
                    i += 1
            else:
                i += 1
        return result
    else:
        return row['relevant_da']
    


In [91]:
df['relevant'] = df.apply(final_relevant, axis=1)

In [92]:
df['relevant'].sum()

488

In [93]:
df.head()

,title,location,country,employment_type,company_name,employee_qty,company_field,skills,job_description,applicants,indicator,city,changed_ind,hard_skills,relevant_jun,relevant_da,relevant
0,data analyst,Basel,Switzerland,On-site,PharmiWeb.Jobs: Global Life Science Jobs,11-50 employees,staffing and recruiting,None,What You Will Achi...,47.0,0,Basel,0,"[data mining, excel, sap, sas, spss, sql, stat...",1,1,1
1,data analyst logistics,Coventry,United Kingdom,On-site,Resolute Recruitment,not specified,not specified,None,,NaN,0,Coventry,0,[],1,1,1
2,data analyst logistics,Coventry,United Kingdom,On-site,Resolute Recruitment,not specified,not specified,None,Data Analyst - Lo...,NaN,0,Coventry,0,[wfh],1,1,1
3,data analyst (space & planning),South Molton,United Kingdom,On-site,Mole Valley Farmers,not specified,not specified,None,Salary: To b...,NaN,0,South Molton,0,[excel],1,1,1
4,data analyst,Lugano,Switzerland,On-site,FORFIRM,not specified,not specified,None,FORFIRM is p...,NaN,0,Lugano,0,"[aws, data warehouse, etl, gcp, oracle, oracle...",1,1,1


Итого, после фильтра осталось 488 подходящих вакансий.  

**Финальные датасеты для основной визуализации**  
Выделим только необходимые вакансии в отдельный датасет. Оставим только необходимые нам для визуализации столбцы: `title`, `country`, `city`, `company_name`, `employment_type`, `employee_qty`, `company_field`. Хард-скиллы будем добавлять в отдельный датасет, т.к. там после explode будут дублироваться строки по другим столбцам.

In [94]:
final_vacancies = df[df['relevant'] == 1][['title', 'country', 'city', 'company_name', 'employment_type', 'employee_qty', 'company_field']].reset_index(drop=True)
final_vacancies.head()

,title,country,city,company_name,employment_type,employee_qty,company_field
0,data analyst,Switzerland,Basel,PharmiWeb.Jobs: Global Life Science Jobs,On-site,11-50 employees,staffing and recruiting
1,data analyst logistics,United Kingdom,Coventry,Resolute Recruitment,On-site,not specified,not specified
2,data analyst logistics,United Kingdom,Coventry,Resolute Recruitment,On-site,not specified,not specified
3,data analyst (space & planning),United Kingdom,South Molton,Mole Valley Farmers,On-site,not specified,not specified
4,data analyst,Switzerland,Lugano,FORFIRM,On-site,not specified,not specified


In [95]:
# выгрузим файл в csv
final_vacancies.to_csv('final_vacancies.csv', index_label = 'vacancy_id')

Теперь сделаем датасет для хард скиллов (нам оттуда понадобятся только сами скиллы и наименования вакансий для счёта строк, где встречаются скиллы)

In [96]:
vacancies_skills = df[df['relevant'] == 1][['title', 'hard_skills']].reset_index(drop=True)

In [97]:
vacancies_skills = vacancies_skills.explode('hard_skills')

In [98]:
vacancies_skills

,title,hard_skills
0,data analyst,data mining
0,data analyst,excel
0,data analyst,sap
0,data analyst,sas
0,data analyst,spss
...,...,...
486,stage | data analyst,ai
486,stage | data analyst,excel
486,stage | data analyst,power point
486,stage | data analyst,powerpoint


На текущем этапе нужно дополнительно обработать столбец hard_skills от неявных дубликатов (здесь одни и теже названия встречаются с пробелами, без или с другими символами). Проще всего можно это исправить удалив все пробелы, дефисы и знаки "слэш". Смотреться будет некарсиво, но максимально быстро и эффективно. Но предварительно проверим нет ли пропусков.

In [99]:
vacancies_skills['hard_skills'].isna().sum()

32

Пропуски предлагаю заменить на 'unknown'

In [100]:
vacancies_skills['hard_skills'] = vacancies_skills['hard_skills'].fillna('unknown')
vacancies_skills['hard_skills'].isna().sum()

0

Теперь дубликаты

In [101]:
vacancies_skills['hard_skills'].nunique()

151

In [102]:
# заменим "/", ".", "-" и пробелы.
vacancies_skills['hard_skills'] = vacancies_skills['hard_skills'].apply(lambda x: x.replace('/', ''))
vacancies_skills['hard_skills'] = vacancies_skills['hard_skills'].apply(lambda x: x.replace('-', ''))
vacancies_skills['hard_skills'] = vacancies_skills['hard_skills'].apply(lambda x: x.replace('.', ''))
vacancies_skills['hard_skills'] = vacancies_skills['hard_skills'].apply(lambda x: x.replace(' ', ''))

In [103]:
vacancies_skills['hard_skills'].nunique()

147

Таким образом, количество всех скиллов снизилось с 151 до 147.  
Далее сохраним в csv

In [104]:
#  сохраним в csv
vacancies_skills.to_csv('vacancies_skills.csv', index_label = 'vacancy_id')

## Визуализации
собран дашборд в Tableau
https://public.tableau.com/app/profile/marat.pshikhachev/viz/linkedIn_dashboard/Dashboard_LinkedIn?publish=yes

## Итого
- На вход было получен датасет, который содержит 10 столбцов (наименование вакансий, локация, страна, тип занятости, название компании, количество сотрудников, сфера деятельности, скиллы, описание вакансии, количество кандидатов)
- было обнаружено 112 полных дубликатов – все удалены
- в столбце с вакансиями всё переведено в нижний регистр, для упрощения отбора.
- столбцы `location` и `country` обработаны на предмет неявных дубликатов. В результате: из 72 записей исходных стран получилось 29 корректных, а из 428 записей уникальных локаций – 375 записей уникальных городов
  - в графе тип занятости есть 67 строк, где не указан тип занятости. Обработать данную ситуацию невозможно – оставлено как есть
- в столбце ` company_name` обнаружено 2 компании без названий – заменено на “ not specified”.
- в столбце с информацией о количестве сотрудников удалена лишняя информация, а также в 4 строках заполнены пропуски по информации с других строк тех же компаний.
- в столбце ` company_field` проведена аналогичная операция (удалено 5 пропусков)
- из столбцов “skills”` и “job_description” выделены хард-скиллы в отдельный столбец “hard_skills”
- в столбце “applicants” обнаружено 159 пропусков оставлены без изменений.

- отобраны только релевантные вакансии (data analyst и BI analyst, в обоих случаях junior) – получилось 488 строк
- по итогу сформированы два датасета: `final_vacancies` и `vacancies_skills` (выгружены в формате CSV). В датасете `vacancies_skills` скиллы дополнительно обработаны от дубликатов (итоговое общее количество скиллов в датасете 147).
- сделан дашборд
